In [10]:
import pyNeuroChem as pyc
import hdnntools as hdt
import numpy as np
import time

In [11]:
mol = hdt.readxyz2('/home/jujuman/Scratch/Research/conjugate_grad/Acetaminophen_opt.xyz')
N = 1000

wkdir = '/home/jujuman/Scratch/Research/conjugate_grad/arc9/'
cns = wkdir + 'rHCNO-4.6R_16-3.1A_a4-8.params'
sae = wkdir + 'sae_linfit.dat'
nnf = wkdir + 'train0/networks/'

In [12]:
nc_mol = pyc.molecule(cns, sae, nnf, 0)
nc_cof = pyc.conformers(cns, sae, nnf, 0)

In [13]:
Xl = np.stack([mol[0][0] for i in range(N)])
S = list(mol[1])
Xl.shape

(1000, 20, 3)

In [14]:
X = Xl+np.random.uniform(-0.1,0.1,size=Xl.shape)
X = np.asarray(X, dtype=np.float32)
#np.abs(X[0]-X[1]).mean()

In [6]:
for i,x in enumerate(X):
    nc_mol.setMolecule(x,S)
    
    print(i)
    
    xn = np.array(x,np.float32)
    hn = np.zeros(xn.shape,dtype=np.float32)
    fn = np.zeros(xn.shape,dtype=np.float32)
    for i in range(3):
        nc_mol.setCoordinates(xn)
        e = nc_mol.energy().copy()
        f = nc_mol.force().copy()
        
        if i != 0:
            gamma = np.power(np.linalg.norm(f),2)/np.power(np.linalg.norm(fn),2)
        else:
            gamma = 0
        fn=f
        
        hn = f + gamma * hn
        xn = xn+0.002*hn
        
        print('  -',i,e[0], hdt.hatokcal*np.abs(f).sum(),hdt.hatokcal*np.max(f))
        if hdt.hatokcal*np.max(f) < 0.01:
            break

0
  - 0 -40.4807318459 549.036262887 60.8376848012
  - 1 -40.4808914597 546.880196985 60.7681863199
  - 2 -40.4812068726 542.596004834 60.6273052075
1
  - 0 -40.4767952571 863.987315677 138.643387573
  - 1 -40.4772801335 852.420599049 136.577171777
  - 2 -40.4782065834 830.062818401 132.591591667
2
  - 0 -40.4645695473 924.79544317 189.929526509
  - 1 -40.4650248001 917.371425114 188.270689163
  - 2 -40.4659118215 902.828742842 185.026416836
3
  - 0 -40.468656973 912.003570939 118.528399825
  - 1 -40.4691506336 901.400416974 117.004594779
  - 2 -40.470100754 881.453749732 114.057503849
4
  - 0 -40.4777860577 780.69184535 148.319941874
  - 1 -40.4781617905 770.904439452 146.478739391
  - 2 -40.478883491 751.968536803 142.908850433
5
  - 0 -40.4576186846 1035.40392417 213.540271502
  - 1 -40.4581594924 1027.79199606 211.574397206
  - 2 -40.4592129821 1012.86436748 207.717173053
6
  - 0 -40.4863160188 561.682228047 96.6047123483
  - 1 -40.4864513586 559.258023773 96.044152045
  - 2 -40.48

In [15]:
hdt.writexyzfile('/home/jujuman/Research/conjugate_grad/verify1.xyz',X,S)

In [18]:
nc_cof.setConformers(X,S)

Nm = X.shape[0]
mask = np.full(Nm,True,dtype=bool)
Xn = np.array(X,np.float32)
Hn = np.zeros(Xn.shape,dtype=np.float32)
Fn = np.zeros(Xn.shape,dtype=np.float32)

start_time = time.time()
for i in range(10000):
    Nm = np.bincount(mask)[1]
    idx = np.argsort(np.invert(mask))
    
    mask = mask[idx]
    Xn = Xn[idx]
    Hn = Hn[idx]
    Fn = Fn[idx]

    start_time1 = time.time()
    nc_cof.setConformers(np.asarray(Xn[0:Nm],dtype=np.float32),S)
    E = nc_cof.energy().copy()
    F = -nc_cof.force().copy()
    end_time1 = time.time()
    #print('[pyNeuroChem Time:',end_time1-start_time1,'s]')

    start_time2 = time.time()    
    if i != 0:
        gamma = np.power(np.linalg.norm(F.reshape(Nm,-1),axis=1),2)/np.power(np.linalg.norm(Fn[0:Nm].reshape(Nm,-1),axis=1),2)
    else:
        gamma = np.zeros(Nm)
    
    Fn[0:Nm]=F
    
    Hn[0:Nm] = (F.reshape(Nm,-1).T + gamma * Hn[0:Nm].reshape(Nm,-1).T).T.reshape(Nm,-1,3)
    Xn[0:Nm] = Xn[0:Nm]-0.005*Hn[0:Nm]
        
    #conv = np.where(hdt.hatokcal*np.sum(np.abs(F.reshape(Nm,-1)),axis=1) < 0.25)
    conv = np.where(hdt.hatokcal*np.max(np.abs(F.reshape(Nm,-1)),axis=1) < 0.01)
    
    mask[conv] = False
    end_time2 = time.time()
    
    print('  -',str(i).zfill(4),')',np.bincount(mask)[0],'of',X.shape[0],'completed. ',hdt.hatokcal*np.max(np.abs(Fn.reshape(X.shape[0],-1)),axis=1).mean())
    if np.bincount(mask)[0] == X.shape[0]:
        break;

end_time = time.time()
print('[Complete Optimization Time:',end_time-start_time,'s]')

  - 0000 ) 0 of 1000 completed.  266.708026299
  - 0001 ) 0 of 1000 completed.  258.507701091
  - 0002 ) 0 of 1000 completed.  242.654641668
  - 0003 ) 0 of 1000 completed.  221.083790477
  - 0004 ) 0 of 1000 completed.  196.747680508
  - 0005 ) 0 of 1000 completed.  172.954654534
  - 0006 ) 0 of 1000 completed.  151.336218039
  - 0007 ) 0 of 1000 completed.  132.451528734
  - 0008 ) 0 of 1000 completed.  116.426735829
  - 0009 ) 0 of 1000 completed.  102.991073429
  - 0010 ) 0 of 1000 completed.  91.6699414774
  - 0011 ) 0 of 1000 completed.  82.0860050652
  - 0012 ) 0 of 1000 completed.  73.9870078481
  - 0013 ) 0 of 1000 completed.  67.0700645511
  - 0014 ) 0 of 1000 completed.  61.1547315897
  - 0015 ) 0 of 1000 completed.  56.142864859
  - 0016 ) 0 of 1000 completed.  51.8126209054
  - 0017 ) 0 of 1000 completed.  48.0721018382
  - 0018 ) 0 of 1000 completed.  44.8944760538
  - 0019 ) 0 of 1000 completed.  42.1568577076
  - 0020 ) 0 of 1000 completed.  39.7890470219
  - 0021 ) 0 o

  - 0175 ) 1 of 1000 completed.  11.7066907309
  - 0176 ) 1 of 1000 completed.  15.0038279551
  - 0177 ) 1 of 1000 completed.  12.2215498802
  - 0178 ) 1 of 1000 completed.  11.7268191087
  - 0179 ) 1 of 1000 completed.  12.3275204538
  - 0180 ) 1 of 1000 completed.  12.2303453069
  - 0181 ) 1 of 1000 completed.  10.5752353518
  - 0182 ) 1 of 1000 completed.  10.4949965149
  - 0183 ) 1 of 1000 completed.  10.9438496394
  - 0184 ) 1 of 1000 completed.  12.5948488282
  - 0185 ) 1 of 1000 completed.  11.1436267992
  - 0186 ) 1 of 1000 completed.  11.3657145376
  - 0187 ) 1 of 1000 completed.  11.0380068738
  - 0188 ) 1 of 1000 completed.  11.0080977477
  - 0189 ) 1 of 1000 completed.  10.1263869027
  - 0190 ) 1 of 1000 completed.  9.95447342096
  - 0191 ) 1 of 1000 completed.  10.1504495538
  - 0192 ) 1 of 1000 completed.  9.7987026154
  - 0193 ) 1 of 1000 completed.  9.77412977111
  - 0194 ) 1 of 1000 completed.  10.8689114351
  - 0195 ) 1 of 1000 completed.  11.6308186334
  - 0196 ) 1 o

  - 0352 ) 1 of 1000 completed.  5.0247717548
  - 0353 ) 1 of 1000 completed.  4.62451993751
  - 0354 ) 1 of 1000 completed.  4.83170074265
  - 0355 ) 1 of 1000 completed.  4.58696697198
  - 0356 ) 1 of 1000 completed.  4.54031322357
  - 0357 ) 1 of 1000 completed.  4.54737674017
  - 0358 ) 1 of 1000 completed.  4.84518813501
  - 0359 ) 1 of 1000 completed.  5.1137767977
  - 0360 ) 1 of 1000 completed.  4.9716380269
  - 0361 ) 1 of 1000 completed.  4.94560005739
  - 0362 ) 1 of 1000 completed.  5.18555566137
  - 0363 ) 1 of 1000 completed.  4.95705573541
  - 0364 ) 1 of 1000 completed.  5.46251868065
  - 0365 ) 1 of 1000 completed.  5.36056536706
  - 0366 ) 1 of 1000 completed.  5.92362228582
  - 0367 ) 1 of 1000 completed.  6.00743890358
  - 0368 ) 1 of 1000 completed.  5.81644372902
  - 0369 ) 1 of 1000 completed.  5.66071441684
  - 0370 ) 1 of 1000 completed.  5.62429609057
  - 0371 ) 1 of 1000 completed.  5.59878000252
  - 0372 ) 1 of 1000 completed.  5.70279863426
  - 0373 ) 1 of 

  - 0529 ) 2 of 1000 completed.  1.90627592935
  - 0530 ) 2 of 1000 completed.  1.60022342423
  - 0531 ) 2 of 1000 completed.  1.5368498911
  - 0532 ) 2 of 1000 completed.  1.60823266836
  - 0533 ) 2 of 1000 completed.  1.73120544463
  - 0534 ) 2 of 1000 completed.  1.90228263031
  - 0535 ) 2 of 1000 completed.  2.37438855453
  - 0536 ) 2 of 1000 completed.  2.36917324637
  - 0537 ) 2 of 1000 completed.  1.78945790632
  - 0538 ) 2 of 1000 completed.  1.96927353835
  - 0539 ) 2 of 1000 completed.  2.04012113874
  - 0540 ) 2 of 1000 completed.  2.23633673509
  - 0541 ) 2 of 1000 completed.  3.04162920191
  - 0542 ) 2 of 1000 completed.  2.06752736691
  - 0543 ) 2 of 1000 completed.  1.94704623859
  - 0544 ) 2 of 1000 completed.  1.78128049714
  - 0545 ) 2 of 1000 completed.  1.91921221957
  - 0546 ) 2 of 1000 completed.  2.08673514701
  - 0547 ) 2 of 1000 completed.  1.78174043075
  - 0548 ) 2 of 1000 completed.  1.73183778029
  - 0549 ) 3 of 1000 completed.  1.96845769678
  - 0550 ) 3 o

  - 0705 ) 8 of 1000 completed.  0.28981824036
  - 0706 ) 8 of 1000 completed.  0.302131034172
  - 0707 ) 8 of 1000 completed.  0.335426710014
  - 0708 ) 8 of 1000 completed.  0.428715505587
  - 0709 ) 8 of 1000 completed.  0.439589326682
  - 0710 ) 8 of 1000 completed.  0.694163784543
  - 0711 ) 8 of 1000 completed.  0.683910756937
  - 0712 ) 8 of 1000 completed.  0.638404797105
  - 0713 ) 8 of 1000 completed.  0.683553168783
  - 0714 ) 8 of 1000 completed.  0.520809394229
  - 0715 ) 8 of 1000 completed.  0.65526096939
  - 0716 ) 8 of 1000 completed.  0.582619474523
  - 0717 ) 8 of 1000 completed.  0.660045199374
  - 0718 ) 8 of 1000 completed.  0.492641127073
  - 0719 ) 8 of 1000 completed.  0.482994355665
  - 0720 ) 8 of 1000 completed.  0.722705017238
  - 0721 ) 8 of 1000 completed.  0.645022040766
  - 0722 ) 8 of 1000 completed.  0.677122207
  - 0723 ) 8 of 1000 completed.  0.534997060352
  - 0724 ) 8 of 1000 completed.  0.525062246025
  - 0725 ) 8 of 1000 completed.  0.5020836824

...

  - 2768 ) 916 of 1000 completed.  0.027941608224
  - 2769 ) 916 of 1000 completed.  0.0287843008687
  - 2770 ) 916 of 1000 completed.  0.0297116330587
  - 2771 ) 916 of 1000 completed.  0.0305185349279
  - 2772 ) 916 of 1000 completed.  0.0308166384383
  - 2773 ) 916 of 1000 completed.  0.0302768707173
  - 2774 ) 916 of 1000 completed.  0.0292001477665
  - 2775 ) 916 of 1000 completed.  0.0282694049744
  - 2776 ) 917 of 1000 completed.  0.0275409127089
  - 2777 ) 917 of 1000 completed.  0.027035271554
  - 2778 ) 919 of 1000 completed.  0.0267020461478
  - 2779 ) 919 of 1000 completed.  0.0264943167426
  - 2780 ) 919 of 1000 completed.  0.0263707999867
  - 2781 ) 919 of 1000 completed.  0.026294659552
  - 2782 ) 919 of 1000 completed.  0.0262331705523
  - 2783 ) 919 of 1000 completed.  0.0261683759624
  - 2784 ) 919 of 1000 completed.  0.0260796478037
  - 2785 ) 919 of 1000 completed.  0.0259559392871
  - 2786 ) 921 of 1000 completed.  0.0257822565476
  - 2787 ) 921 of 1000 completed. 

  - 2937 ) 961 of 1000 completed.  0.0362431254189
  - 2938 ) 961 of 1000 completed.  0.038467856099
  - 2939 ) 962 of 1000 completed.  0.0417252823735
  - 2940 ) 962 of 1000 completed.  0.0467391047717
  - 2941 ) 962 of 1000 completed.  0.0543070754354
  - 2942 ) 963 of 1000 completed.  0.0613897818074
  - 2943 ) 963 of 1000 completed.  0.0516958829106
  - 2944 ) 963 of 1000 completed.  0.0398222075338
  - 2945 ) 963 of 1000 completed.  0.0354457065757
  - 2946 ) 963 of 1000 completed.  0.0332928222591
  - 2947 ) 963 of 1000 completed.  0.0323009443115
  - 2948 ) 963 of 1000 completed.  0.0322305133251
  - 2949 ) 963 of 1000 completed.  0.033155341211
  - 2950 ) 964 of 1000 completed.  0.035491904938
  - 2951 ) 964 of 1000 completed.  0.040345837674
  - 2952 ) 964 of 1000 completed.  0.0501856391122
  - 2953 ) 964 of 1000 completed.  0.0693791056421
  - 2954 ) 964 of 1000 completed.  0.0912648703653
  - 2955 ) 966 of 1000 completed.  0.0577513725671
  - 2956 ) 966 of 1000 completed.  

  - 3109 ) 983 of 1000 completed.  0.0196089732806
  - 3110 ) 984 of 1000 completed.  0.0194130234647
  - 3111 ) 984 of 1000 completed.  0.0192558686761
  - 3112 ) 984 of 1000 completed.  0.0191391491542
  - 3113 ) 984 of 1000 completed.  0.0190656591271
  - 3114 ) 984 of 1000 completed.  0.0190401446736
  - 3115 ) 984 of 1000 completed.  0.019069514888
  - 3116 ) 984 of 1000 completed.  0.0191637185011
  - 3117 ) 985 of 1000 completed.  0.0193343627458
  - 3118 ) 986 of 1000 completed.  0.0195998212702
  - 3119 ) 987 of 1000 completed.  0.0199869291661
  - 3120 ) 987 of 1000 completed.  0.0205357005111
  - 3121 ) 987 of 1000 completed.  0.0213082794878
  - 3122 ) 987 of 1000 completed.  0.0224073950727
  - 3123 ) 987 of 1000 completed.  0.0240053739829
  - 3124 ) 987 of 1000 completed.  0.0264062620295
  - 3125 ) 987 of 1000 completed.  0.0301236150641
  - 3126 ) 987 of 1000 completed.  0.035887128423
  - 3127 ) 987 of 1000 completed.  0.0428939637385
  - 3128 ) 987 of 1000 completed.

  - 3351 ) 994 of 1000 completed.  0.034770991788
  - 3352 ) 994 of 1000 completed.  0.0336011712727
  - 3353 ) 994 of 1000 completed.  0.032209417359
  - 3354 ) 994 of 1000 completed.  0.0313173349052
  - 3355 ) 994 of 1000 completed.  0.0308656241802
  - 3356 ) 994 of 1000 completed.  0.0306962925835
  - 3357 ) 994 of 1000 completed.  0.0306888184798
  - 3358 ) 994 of 1000 completed.  0.0307555283944
  - 3359 ) 994 of 1000 completed.  0.0308239824187
  - 3360 ) 994 of 1000 completed.  0.0308395036257
  - 3361 ) 994 of 1000 completed.  0.0307616487585
  - 3362 ) 994 of 1000 completed.  0.0305606515255
  - 3363 ) 994 of 1000 completed.  0.0302186119732
  - 3364 ) 994 of 1000 completed.  0.0297357720854
  - 3365 ) 994 of 1000 completed.  0.0291415625725
  - 3366 ) 994 of 1000 completed.  0.0284870191901
  - 3367 ) 994 of 1000 completed.  0.0278259468106
  - 3368 ) 994 of 1000 completed.  0.0271983298344
  - 3369 ) 994 of 1000 completed.  0.0266259399562
  - 3370 ) 994 of 1000 completed.

  - 3640 ) 998 of 1000 completed.  0.0170351832348
  - 3641 ) 998 of 1000 completed.  0.0168005981921
  - 3642 ) 998 of 1000 completed.  0.0165536525707
  - 3643 ) 998 of 1000 completed.  0.016298764858
  - 3644 ) 998 of 1000 completed.  0.0160399300696
  - 3645 ) 998 of 1000 completed.  0.0157804458059
  - 3646 ) 998 of 1000 completed.  0.0155306934005
  - 3647 ) 998 of 1000 completed.  0.0152869209631
  - 3648 ) 998 of 1000 completed.  0.0150502801994
  - 3649 ) 998 of 1000 completed.  0.0148229318425
  - 3650 ) 998 of 1000 completed.  0.0146053929614
  - 3651 ) 998 of 1000 completed.  0.0144070598332
  - 3652 ) 998 of 1000 completed.  0.0142200885298
  - 3653 ) 998 of 1000 completed.  0.014044423121
  - 3654 ) 998 of 1000 completed.  0.0138799334835
  - 3655 ) 998 of 1000 completed.  0.0137267212047
  - 3656 ) 998 of 1000 completed.  0.0135844164604
  - 3657 ) 998 of 1000 completed.  0.0134616108174
  - 3658 ) 999 of 1000 completed.  0.0133399443245
  - 3659 ) 999 of 1000 completed.

In [19]:
hdt.writexyzfile('/home/jujuman/Research/conjugate_grad/verify2.xyz',Xn,S)